In [ ]:
import numpy as np
import pandas as pd
from xgboost.sklearn import XGBClassifier
from sklearn import  metrics  
from sklearn.metrics import accuracy_score  
import xgboost as xgb 
from sklearn.model_selection import GridSearchCV
from os.path import exists
import pickle
from sklearn.model_selection import train_test_split     
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import linear_model, preprocessing
from numpy import sort
from sklearn.feature_selection import SelectFromModel
from numpy import loadtxt
import time
#use feature importance for feature selection, with fix for xgboost 1.0.2
 

In [ ]:
filepath1='../input/training/Xtr.csv'
filepath2='../input/training/Xts.csv'
filepath3='../input/training/ytr.csv'
filepath_y_validation='../input/y-validation/yts_XGBoost.csv'
X_tr = np.loadtxt(filepath1, delimiter=",")
X_validation = np.loadtxt(filepath2, delimiter=",")
y_tr = np.loadtxt(filepath3, delimiter=",")
#check_y_validation=np.loadtxt(filepath_y_validation, delimiter=",")
num_features = X_tr.shape[1]
num_samples = X_tr.shape[0]
print('The unique labels: ', np.unique(y_tr))
print('The number of features: ', num_features)
print('The number of samples: ', num_samples)

In [ ]:
# define custom class to fix bug in xgboost 1.0.2
class MyXGBClassifier(XGBClassifier):
	@property
	def coef_(self):
		return None

In [ ]:
xgb = MyXGBClassifier(learning_rate=0.1,n_estimators=350,max_depth=8,min_child_weight=1,gamma=0.1,subsample=0.8,   
                     colsample_bytree=0.8,objective= 'multi:softprob',num_class=26,seed=27)
xgb.fit(X_tr,y_tr,eval_metric='mlogloss')

print(xgb.feature_importances_)
# plot
plt.bar(range(len(xgb.feature_importances_)), xgb.feature_importances_)
plt.show()

In [ ]:
thresholds = sort(xgb.feature_importances_)

selection = SelectFromModel(xgb, threshold=thresholds[8], prefit=True)
select_X_tr = selection.transform(X_tr)
    
	# train model
selection_model = MyXGBClassifier(learning_rate=0.1,n_estimators=350,max_depth=8,min_child_weight=1,gamma=0.1,subsample=0.8,   
                     colsample_bytree=0.8,objective= 'multi:softprob',num_class=26,seed=27)
selection_model.fit(select_X_tr, y_tr,eval_metric='mlogloss')
    
    #the shape size is determined by the feature selection from 16 to 1.
#print ('the size of select_X_test.shape',select_X_test.shape)
    
predictions = selection_model.predict(select_X_tr)
accuracy = accuracy_score(y_tr, predictions)
#print("Thresh=%.7f, n=%d, Accuracy: %.2f%%" % (thresh, select_X_train.shape[1], accuracy*100.0))

In [ ]:
#selection = SelectFromModel(xgb, threshold=thresholds[8], prefit=True)
#select_X_train = selection.transform(X_train)
select_X_validation = selection.transform(X_validation)
y_validation=selection_model.predict(select_X_validation)    
y_validation=[round(value) for value in y_validation]

#accuracy1 = accuracy_score(check_y_validation,y_validation)
#print ('check the accuracy of output=%f'%(accuracy1*100.0))

In [ ]:
yts_path = ('yts_XGBoost'+format(time.time())+'.csv')
df = pd.DataFrame(data={'Id':np.arange(len(y_validation)), 
                        'Label':np.int64(y_validation)})
df.to_csv(yts_path, index=False)